<a href="https://colab.research.google.com/github/Daishinkan002/Relation-Extraction/blob/main/Supervised_RE/Supervised_RE2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/My\ Drive/DATA/RE_NYT/

/content/gdrive/My Drive/DATA/RE_NYT


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import torch
from torch import nn
from collections import Counter 
from sklearn.preprocessing import OneHotEncoder
from torchvision import datasets, transforms
import torch
import torch.nn.functional as F 
from torch import optim 
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer

In [4]:
train_df = pd.read_json('raw_train.json', lines = True)
test_df = pd.read_json('raw_test.json', lines = True)
valid_df = pd.read_json('raw_valid.json', lines = True)

In [5]:
train_df.head()

,sentText,articleId,relationMentions,entityMentions,sentId
0,Massachusetts ASTON MAGNA Great Barrington ; a...,/m/vinci8/data1/riedel/projects/relation/kb/ny...,"[{'em1Text': 'Annandale-on-Hudson', 'em2Text':...","[{'start': 1, 'label': 'ORGANIZATION', 'text':...",1
1,North Carolina EASTERN MUSIC FESTIVAL Greensbo...,/m/vinci8/data1/riedel/projects/relation/kb/ny...,"[{'em1Text': 'North Carolina', 'em2Text': 'Gre...","[{'start': 0, 'label': 'LOCATION', 'text': 'No...",2
2,It will be the final movie credited to Debra H...,/m/vinci8/data1/riedel/projects/relation/kb/ny...,"[{'em1Text': 'Debra Hill', 'em2Text': 'Haddonf...","[{'start': 0, 'label': 'LOCATION', 'text': 'De...",1
3,In a 3-0 victory over the Crew on Saturday in ...,/m/vinci8/data1/riedel/projects/relation/kb/ny...,"[{'em1Text': 'Ohio', 'em2Text': 'Columbus', 'l...","[{'start': 0, 'label': 'LOCATION', 'text': 'Co...",1
4,"The United States ambassador to Mexico , Tony ...",/m/vinci8/data1/riedel/projects/relation/kb/ny...,"[{'em1Text': 'Mexico', 'em2Text': 'Nuevo Lared...","[{'start': 0, 'label': 'LOCATION', 'text': 'Un...",1


In [6]:
train_relations = [rel_sets[0]['label'] for rel_sets in train_df['relationMentions']]
test_relations = [rel_sets[0]['label'] for rel_sets in test_df['relationMentions']]
valid_relations = [rel_sets[0]['label'] for rel_sets in valid_df['relationMentions']]
train_df['relations'] = train_relations
test_df['relations'] = test_relations
valid_df['relations'] = valid_relations

In [7]:
train_df['relations']

0                   /location/location/contains
1                   /location/location/contains
2                 /people/person/place_of_birth
3                   /location/location/contains
4                   /location/location/contains
                          ...                  
56191               /location/location/contains
56192                 /location/country/capital
56193                  /business/person/company
56194    /location/neighborhood/neighborhood_of
56195    /location/neighborhood/neighborhood_of
Name: relations, Length: 56196, dtype: object

In [8]:
def get_label_dictionaries(df):
  dictionary = {}
  rev_dictionary = {}
  for i, rel in enumerate(df.relations.unique()):
    #map relation to index
    dictionary[rel] = i
    #map index to relation
    rev_dictionary[i] = rel
  return dictionary, rev_dictionary

In [9]:
dictionary, rev_dictionary = get_label_dictionaries(train_df)
for key, value in dictionary.items():
  print(value, "<--", key)

0 <-- /location/location/contains
1 <-- /people/person/place_of_birth
2 <-- /business/person/company
3 <-- /people/person/place_lived
4 <-- /location/administrative_division/country
5 <-- /people/person/religion
6 <-- /people/person/nationality
7 <-- /people/person/children
8 <-- /location/country/capital
9 <-- /business/company/place_founded
10 <-- /people/deceased_person/place_of_death
11 <-- /location/country/administrative_divisions
12 <-- /business/company/founders
13 <-- /location/neighborhood/neighborhood_of
14 <-- /business/company/advisors
15 <-- /people/ethnicity/geographic_distribution
16 <-- /sports/sports_team_location/teams
17 <-- /sports/sports_team/location
18 <-- /business/company_shareholder/major_shareholder_of
19 <-- /business/company/major_shareholders
20 <-- /people/person/ethnicity
21 <-- /people/ethnicity/people
22 <-- /people/person/profession
23 <-- /business/company/industry


In [10]:
encoded_training_relations = [dictionary[rel] for rel in train_df['relations']]
encoded_testing_relations = [dictionary[rel] for rel in test_df['relations']]
encoded_valid_relations = [dictionary[rel] for rel in valid_df['relations']]
train_df['encoded_relations'] = encoded_training_relations
test_df['encoded_relations'] = encoded_testing_relations
valid_df['encoded_relations'] = encoded_valid_relations

In [11]:
def find_max_len(x, max_len):
  if(max_len < len(x.split())):
    max_len = len(x.split())
  return max_len

In [12]:
max_len = 0
for x in train_df['sentText']:
  max_len = find_max_len(x, max_len)
print("Max Length of Sentence in DF = ", max_len)

Max Length of Sentence in DF =  100


In [13]:
print("Number of Unique Relations in Training_Data= ", len(train_df.relations.unique()))

Number of Unique Relations in Training_Data=  24


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [15]:
sequence_length = 100
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 24
batch_size = 64
num_epochs = 10
learning_rate = 0.01

In [16]:
training_data = [sentence.lower().split() for sentence in train_df['sentText']]
testing_data = [sentence.lower().split() for sentence in test_df['sentText']]
valid_data = [sentence.lower().split() for sentence in valid_df['sentText']]
y_train = [label for label in train_df['encoded_relations']]
y_test = [label for label in test_df['encoded_relations']]
y_valid = [label for label in valid_df['encoded_relations']]

In [17]:
onehot_encoder = OneHotEncoder(sparse=False)
reshaped_y_train = np.array(y_train).reshape(len(y_train), 1)
y_train_encoded = onehot_encoder.fit_transform(reshaped_y_train)
reshaped_y_test = np.array(y_test).reshape(len(y_test), 1)
y_test_encoded = onehot_encoder.fit_transform(reshaped_y_test)
reshaped_y_valid = np.array(y_valid).reshape(len(y_valid), 1)
y_valid_encoded = onehot_encoder.fit_transform(reshaped_y_valid)

In [18]:
dictionary

{'/business/company/advisors': 14,
 '/business/company/founders': 12,
 '/business/company/industry': 23,
 '/business/company/major_shareholders': 19,
 '/business/company/place_founded': 9,
 '/business/company_shareholder/major_shareholder_of': 18,
 '/business/person/company': 2,
 '/location/administrative_division/country': 4,
 '/location/country/administrative_divisions': 11,
 '/location/country/capital': 8,
 '/location/location/contains': 0,
 '/location/neighborhood/neighborhood_of': 13,
 '/people/deceased_person/place_of_death': 10,
 '/people/ethnicity/geographic_distribution': 15,
 '/people/ethnicity/people': 21,
 '/people/person/children': 7,
 '/people/person/ethnicity': 20,
 '/people/person/nationality': 6,
 '/people/person/place_lived': 3,
 '/people/person/place_of_birth': 1,
 '/people/person/profession': 22,
 '/people/person/religion': 5,
 '/sports/sports_team/location': 17,
 '/sports/sports_team_location/teams': 16}

In [19]:
#Padding Encoded Doc
dup_y_valid_encoded = np.zeros((y_valid_encoded.shape[0], num_classes))
dup_y_valid_encoded[:y_valid_encoded.shape[0],:y_valid_encoded.shape[1]] = y_valid_encoded
y_valid_encoded = dup_y_valid_encoded


dup_y_test_encoded = np.zeros((y_test_encoded.shape[0], num_classes))
dup_y_test_encoded[:y_test_encoded.shape[0],:y_test_encoded.shape[1]] = y_test_encoded
y_test_encoded = dup_y_test_encoded

In [20]:
print(y_train_encoded[:10])
print(y_test_encoded[:10])
print(y_valid_encoded[:10])

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0

In [21]:
all_words = [] # Contains all words in the vocabulary

for i, sent in tqdm(enumerate(training_data)):
  refined_sentence = []
  for word in sent:
      all_words.append(word)

count_words = Counter(all_words)
total_words=len(all_words)
sorted_words=count_words.most_common(total_words)

56196it [00:00, 233615.98it/s]


In [22]:
vocab_dictionary ={w: i+1 for i,(w,c) in enumerate(sorted_words)} # Contains a mapper of words to integers of sentences

In [23]:
def get_encoded_doc(dataset, vocab_dictionary):
  encoded_doc = []
  for sentence in dataset:
    encoded_sentence=[]
    for word in sentence:
      if word not in vocab_dictionary.keys():
        encoded_sentence.append(0)
      else:
        encoded_sentence.append(vocab_dictionary[word])
    encoded_doc.append(encoded_sentence)
  return encoded_doc

In [24]:
training_encoded_doc = get_encoded_doc(training_data, vocab_dictionary)
testing_encoded_doc = get_encoded_doc(testing_data, vocab_dictionary)
valid_encoded_doc = get_encoded_doc(valid_data, vocab_dictionary)

In [25]:
def perform_padding(encoded_doc, max_len):
  for i, sent in enumerate(encoded_doc):
    zeros = [0 for i in range(100-len(sent))]
    if(len(zeros)):
      encoded_doc[i] += zeros
  return encoded_doc

In [26]:
X_train = perform_padding(training_encoded_doc, max_len)
X_test = perform_padding(testing_encoded_doc, max_len)
X_valid = perform_padding(valid_encoded_doc, max_len)

In [27]:
train_data=TensorDataset(torch.LongTensor(X_train), torch.FloatTensor(y_train_encoded))
test_data=TensorDataset(torch.LongTensor(X_test), torch.FloatTensor(y_test_encoded))
valid_data = TensorDataset(torch.LongTensor(X_valid), torch.FloatTensor(y_valid_encoded))

In [28]:
batch_size = 128

In [29]:
train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last = True)
test_loader=DataLoader(test_data, batch_size=batch_size, shuffle=True, drop_last = True)
valid_loader=DataLoader(valid_data, batch_size=batch_size, shuffle=True, drop_last = True)

In [30]:
vocab_size = len(vocab_dictionary)+1 # +1 for the 0 padding
output_size = 24
embedding_dim = 300
hidden_dim = 200
n_layers = 2
sequence_length = 100

In [31]:

class RE_LSTM(nn.Module):
    """
    The BIDirectional LSTM model for RE.
    """
    def __init__(self, vocab_size, output_size,  num_classes, embedding_dim, hidden_dim, n_layers, drop_prob=0.6):    
        """
        Initialize the model by setting up the layers
        """
        super().__init__()
        self.output_size=output_size
        self.n_layers=n_layers
        self.hidden_dim=hidden_dim
        
        #Embedding and LSTM layers
        self.embedding=nn.Embedding(vocab_size, embedding_dim)
        self.lstm=nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        #dropout layer
        self.dropout=nn.Dropout(0.3)
        
        #Linear and sigmoid layer
        self.final=nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size=x.size(0)
        #print("Batch_Size = ", batch_size)
        embedd=self.embedding(x)
        #print("Embedding_Shape = ", embedd.shape)
        #embedd = embedd.permute(1,0,2)
        lstm_out, hidden=self.lstm(embedd, hidden)   # Shape -> batch_size, word_length, hidden_dim   torch.Size([128, 100, 128])

        #stack up the lstm output
        #print("----------------\nBefore :- ", lstm_out.shape)
        #lstm_out=lstm_out.contiguous().view(-1, self.hidden_dim)
        #print("After :- ", lstm_out.shape)  -> torch.Size([12800, 128])
        
        #print("LSTM Output Dimension = ", lstm_out.shape)
        out=self.dropout(lstm_out)[:,-1,:]
        #print("Dropout Shape = ", out.shape)
        out = self.final(out)
        #out = out.view(batch_size, -1)
        #print("Final Output = ", out.shape) #--> torch.Size([128, 2400])
        return out, hidden
        '''
        Batch_Size =  128
        LSTM Output Dimension =  torch.Size([12800, 128])
        Dropout Dimension =  torch.Size([12800, 128])
        Final Output =  torch.Size([128, 2400])
        torch.Size([128, 24]) torch.Size([128, 2400])

        '''
    
    def init_hidden(self, batch_size):
        """Initialize Hidden STATE"""
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (cuda_availability_status):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [32]:
model = RE_LSTM(vocab_size, output_size, num_classes, embedding_dim, hidden_dim, n_layers)
print(model)

RE_LSTM(
  (embedding): Embedding(73188, 300)
  (lstm): LSTM(300, 200, num_layers=2, batch_first=True, dropout=0.6)
  (dropout): Dropout(p=0.3, inplace=False)
  (final): Linear(in_features=200, out_features=24, bias=True)
)


In [33]:
lr=0.001

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


# training params

epochs = 3 # 3-4 is approx where I noticed the validation loss stop decreasing

counter = 0
modulo = 100
clip=5 # gradient clipping

# move model to GPU, if available
cuda_availability_status = torch.cuda.is_available()
if(cuda_availability_status):
    model.cuda()

model.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = model.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        #print("Inputs, Labels shape = ", inputs.shape, labels.shape)
        counter += 1
        #print("Counter = ", counter)

        if(cuda_availability_status):
            inputs=inputs.to(device)
            labels=labels.to(device)
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        model.zero_grad()

        #print(counter)
        # get the output from the model
        #print(h[0].shape)
        output, h = model(inputs, h)
      
        #print("Output  Shape = ", output.shape," Labels shape = ", labels.shape)
        loss = criterion(output, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % modulo == 0:
            # Get validation loss
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])
                if cuda_availability_status:
                  inputs, labels = inputs.cuda(), labels.cuda()
                #print("Hiya")
                
                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)

                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Epoch: 1/3... Step: 100... Loss: 0.105372... Val Loss: 0.108385
Epoch: 1/3... Step: 200... Loss: 0.110480... Val Loss: 0.108237
Epoch: 1/3... Step: 300... Loss: 0.118676... Val Loss: 0.108092
Epoch: 1/3... Step: 400... Loss: 0.112773... Val Loss: 0.108156
Epoch: 2/3... Step: 500... Loss: 0.099645... Val Loss: 0.108221
Epoch: 2/3... Step: 600... Loss: 0.104027... Val Loss: 0.108282
Epoch: 2/3... Step: 700... Loss: 0.110474... Val Loss: 0.108350
Epoch: 2/3... Step: 800... Loss: 0.109633... Val Loss: 0.108309
Epoch: 3/3... Step: 900... Loss: 0.112705... Val Loss: 0.108017
Epoch: 3/3... Step: 1000... Loss: 0.105733... Val Loss: 0.108220
Epoch: 3/3... Step: 1100... Loss: 0.107786... Val Loss: 0.108312
Epoch: 3/3... Step: 1200... Loss: 0.106899... Val Loss: 0.108197
Epoch: 3/3... Step: 1300... Loss: 0.120605... Val Loss: 0.108277


In [34]:
print(output[:10])

tensor([[  0.3108,  -3.6267,  -2.3783,  -2.5321,  -3.1476,  -6.3880,  -2.4770,
          -5.0742,  -2.8251,  -4.8943,  -4.3323,  -3.5880,  -5.4585,  -3.1707,
          -7.4531,  -7.0251,  -6.2914,  -6.5530,  -6.5666,  -6.4080,  -8.1619,
          -8.5408,  -9.4548,  -9.8428],
        [  0.3830,  -3.2871,  -2.6384,  -2.4937,  -3.5856,  -7.0690,  -2.2884,
          -4.9211,  -3.1785,  -5.0762,  -4.2539,  -3.6474,  -5.5495,  -3.5059,
          -6.7669,  -7.6402,  -5.8391,  -6.8278,  -6.7473,  -7.1070,  -7.8805,
          -8.4496,  -8.9159,  -9.5202],
        [  0.3079,  -3.9445,  -2.7151,  -2.6186,  -3.3251,  -6.8420,  -2.4928,
          -5.0275,  -2.8664,  -4.9193,  -4.0989,  -3.8304,  -5.0684,  -3.1957,
          -7.0678,  -8.4311,  -6.3991,  -7.1579,  -6.1888,  -6.8528,  -8.5810,
          -9.3179,  -9.4969, -10.0476],
        [  0.4609,  -4.2812,  -2.7577,  -2.8695,  -3.3979,  -7.0176,  -2.4881,
          -5.4728,  -3.4401,  -5.7832,  -4.7836,  -3.9899,  -6.0760,  -3.0659,
          -

In [72]:

test_losses = [] # track loss
num_correct = 0

# init hidden state
model.to(device)
h = model.init_hidden(batch_size)

model.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(cuda_availability_status):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    #print("Inputs shape = ", inputs.shape)
    #inputs = inputs.type(torch.LongTensor).to(device)
    output, h = model(inputs, h)
    
    # calculate loss
    #print(output.squeeze().shape, labels.shape)
    test_loss = criterion(output.squeeze(), labels)
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    #print("Output Sqeeze shape = ", output.squeeze().shape)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    #print(pred.shape, labels.shape)
    correct_tensor = pred.eq(labels.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not cuda_availability_status else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

Test loss: 0.109
Test accuracy: 0.481
